In [89]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

%pip install numpy-financial yfinance pandas numpy matplotlib --quiet
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime




Note: you may need to restart the kernel to use updated packages.


## Group Assignment
### Team Number: 18
### Team Member Names: Poneesh Kumar, Ian Leung, Vivaan Goel
### Team Strategy Chosen: Market Beat(Market Beat, Market Meet, Risk-Free)

In [90]:
filename = "Tickers_Example.csv"
tickers = pd.read_csv(filename, header=None, names=['Ticker'])
tickers.head()

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Poneesh, Ian, Vivaan

In [91]:

# temp
# beta, volume, regularMarketVolume, averageVolume, average(Daily)Volume10days/Day, marketCap, 
# currency, earningsQuarterlyGrowth, financialCurrency, market = us_market

history = {}
def averageDailyVolume(ticker):
    start_date = "2024-10-01"
    end_date = "2025-09-30"

    history[ticker.ticker] = ticker.history(start=start_date, end=end_date)

    daily_volume = history[ticker.ticker]['Volume']

    daily_volume.index = daily_volume.index.tz_localize(None)

    volume_df = daily_volume.to_frame()
    volume_df['Months'] = volume_df.index.to_period('M')

    daily_volume.index = pd.to_datetime(daily_volume.index)
    daily_volume.index = daily_volume.index.strftime('%Y-%m-%d') 
    volume_df.index = pd.to_datetime(volume_df.index)
    volume_df.index = volume_df.index.strftime('%Y-%m-%d') 

    monthly_counts = volume_df.groupby('Months')['Volume'].count()
    valid_months = monthly_counts[monthly_counts >= 18].index
    valid_vol_df = volume_df[volume_df['Months'].isin(valid_months)]

    volumes = []
    
    for volume in valid_vol_df['Volume']:
        volumes.append(volume)
    
    average_daily_vol = np.mean(volumes)
    return average_daily_vol
    



In [137]:
day1 = "2025-10-26"
day15 = "2025-11-15"
def scoring1(ticker):
    start_date = day1
    end_date = day15

    history[ticker.ticker] = ticker.history(start=start_date, end=end_date)

    daily_close = history[ticker.ticker]['Close']
    daily_volume = history[ticker.ticker]['Volume']

    daily_close.index = daily_close.index.tz_localize(None)
    daily_volume.index = daily_volume.index.tz_localize(None)

    daily_close.index = pd.to_datetime(daily_close.index)
    daily_close.index = daily_close.index.strftime('%Y-%m-%d') 
    daily_volume.index = pd.to_datetime(daily_volume.index)
    daily_volume.index = daily_volume.index.strftime('%Y-%m-%d') 
    
    price_change = daily_close.pct_change()
    price_change.drop(index=price_change.index[0], inplace=True)
    volume_change = daily_volume.pct_change()
    volume_change.drop(index=volume_change.index[0], inplace=True)
    
    avg_price_roc = np.mean(price_change)*100
    avg_vol_roc = np.mean(volume_change)*100

    # Calculating the Relative Strength Index
    delta = daily_close.astype(float).diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    window = 7
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    latest_rsi = rsi.iloc[-1]

    # Calculating the simple moving average
    sma_7 = daily_close.astype(float).rolling(window=7).mean().iloc[-1]

    #Calculating the last closing price
    last_close = daily_close.iloc[-1].round(2)

    # Find the difference between the SMA_7 and last closing price as a %
    sma_trend_strength = ((last_close - sma_7)/ last_close) * 100
    

    return avg_price_roc, avg_vol_roc, latest_rsi, sma_7, sma_trend_strength

In [138]:
tickers_df = pd.DataFrame(columns=[
    'Ticker', 'Sector', 'Market Cap', 'Country', 'Market',
    'Avg Daily Vol', 'Beta',
    'Price % ROC', 'Vol % ROC',
    'RSI', 'SMA', 'SMA Trend'
])

row = 0
for ticker_sym in tickers['Ticker']:
    ticker = yf.Ticker(ticker_sym)
    try:
        price_roc, vol_roc, rsi_val, sma_val, sma_trend = scoring1(ticker)
        tickers_df.loc[row] = {
        'Ticker': ticker_sym,
        'Sector': ticker.info['sector'],
        'Market Cap': ticker.info['marketCap'],
        'Country': ticker.info['country'],
        'Market': ticker.info['market'],
        'Avg Daily Vol': averageDailyVolume(ticker),
        'Beta': ticker.info['beta'],
        'Price % ROC': price_roc.round(3),
        'Vol % ROC': vol_roc.round(3),
        'RSI': rsi_val.round(3),
        'SMA': sma_val.round(3),
        'SMA Trend': sma_trend.round(3)
        }

        row+= 1
    except:
        print(f"{ticker_sym} invalid, skipping...")

tickers_df

$AGN: possibly delisted; no timezone found


AGN invalid, skipping...


$CELG: possibly delisted; no timezone found


CELG invalid, skipping...


$MON: possibly delisted; no timezone found


MON invalid, skipping...


$RTN: possibly delisted; no timezone found


RTN invalid, skipping...


,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC,RSI,SMA,SMA Trend
0,AAPL,Technology,4029017227264,United States,us_market,5.384430e+07,1.109,0.104,2.319,60.961,271.604,0.296
1,ABBV,Healthcare,417597620224,United States,us_market,6.362559e+06,0.352,0.155,11.961,92.433,225.709,2.863
2,ABT,Healthcare,222970200064,United States,us_market,6.189108e+06,0.719,0.194,2.740,93.276,127.553,2.326
3,ACN,Technology,156865445888,Ireland,us_market,3.444719e+06,1.262,-0.149,2.505,43.922,244.789,0.172
4,AIG,Financial Services,42082131968,United States,us_market,4.416646e+06,0.622,-0.125,5.790,66.745,77.197,0.711
5,AMZN,Consumer Cyclical,2359223910400,United States,us_market,4.246964e+07,1.368,0.290,7.105,21.933,243.060,-3.566
6,AXP,Financial Services,245569896448,United States,us_market,2.800084e+06,1.147,-0.084,4.552,32.057,366.713,-2.669
7,BA,Industrials,136589156352,United States,us_market,9.557990e+06,1.170,-0.949,9.060,18.238,195.104,-0.300
8,BAC,Financial Services,381902225408,United States,us_market,3.933537e+07,1.304,-0.050,5.118,52.395,53.304,-1.320
9,BB.TO,Technology,3501786880,Canada,ca_market,2.494208e+06,1.023,-0.735,3.305,11.765,6.311,-4.667


In [139]:
filtered_df = tickers_df[(tickers_df["Country"] == 'Canada') | (tickers_df["Country"] == "United States")].reset_index(drop=True)
filtered_df = filtered_df[(filtered_df["Market"] == 'ca_market') | (filtered_df["Market"] == "us_market")].reset_index(drop=True)
filtered_df = filtered_df[filtered_df["Avg Daily Vol"] > 5000].reset_index(drop=True)
filtered_df

,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC,RSI,SMA,SMA Trend
0,AAPL,Technology,4029017227264,United States,us_market,5.384430e+07,1.109,0.104,2.319,60.961,271.604,0.296
1,ABBV,Healthcare,417597620224,United States,us_market,6.362559e+06,0.352,0.155,11.961,92.433,225.709,2.863
2,ABT,Healthcare,222970200064,United States,us_market,6.189108e+06,0.719,0.194,2.740,93.276,127.553,2.326
3,AIG,Financial Services,42082131968,United States,us_market,4.416646e+06,0.622,-0.125,5.790,66.745,77.197,0.711
4,AMZN,Consumer Cyclical,2359223910400,United States,us_market,4.246964e+07,1.368,0.290,7.105,21.933,243.060,-3.566
5,AXP,Financial Services,245569896448,United States,us_market,2.800084e+06,1.147,-0.084,4.552,32.057,366.713,-2.669
6,BA,Industrials,136589156352,United States,us_market,9.557990e+06,1.170,-0.949,9.060,18.238,195.104,-0.300
7,BAC,Financial Services,381902225408,United States,us_market,3.933537e+07,1.304,-0.050,5.118,52.395,53.304,-1.320
8,BB.TO,Technology,3501786880,Canada,ca_market,2.494208e+06,1.023,-0.735,3.305,11.765,6.311,-4.667
9,BIIB,Healthcare,25716908032,United States,us_market,1.555065e+06,0.145,0.785,8.754,92.581,160.261,4.350


In [140]:
def get_score(ticker_df):
    ticker_df['Score'] = (
        # 1: Momentum (75%)
        350 * bounded_score(ticker_df['Price % ROC'], 0, 1.2) +
        250 * bounded_score(ticker_df['RSI'], 40, 80) +
        150 * bounded_score(ticker_df['SMA Trend'], 0.5, -2) + 
        
        # 2: Volume (20%)
        200 * bounded_score(ticker_df['Vol % ROC'], 0, 7) + 

        # 3: Volatility (5%)
        200 * bounded_score(ticker_df['Beta'], 0.5, 1.5)
        )

def bounded_score(col, min, max):
    return np.round(np.clip((col - min) / (max - min), 0, 1), 3)

In [141]:
get_score(filtered_df)
filtered_df = filtered_df.sort_values(by='Score', ascending=False).reset_index(drop=True)
filtered_df

,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC,RSI,SMA,SMA Trend,Score
0,LLY,Healthcare,949975187456,United States,us_market,3.976943e+06,0.389,1.579,7.901,89.718,982.059,4.216,800.00
1,BIIB,Healthcare,25716908032,United States,us_market,1.555065e+06,0.145,0.785,8.754,92.581,160.261,4.350,678.90
2,CAT,Industrials,258875473920,United States,us_market,2.586073e+06,1.562,0.419,10.646,32.427,564.609,-1.909,666.75
3,AMZN,Consumer Cyclical,2359223910400,United States,us_market,4.246964e+07,1.368,0.290,7.105,21.933,243.060,-3.566,608.30
4,BMY,Healthcare,94153580544,United States,us_market,1.261973e+07,0.309,0.548,7.169,51.786,47.666,-2.177,583.70
5,MRK,Healthcare,244183302144,United States,us_market,1.396483e+07,0.319,0.407,11.131,99.883,89.580,3.594,568.65
6,BK,Financial Services,75058782208,United States,us_market,3.981636e+06,1.123,0.194,10.581,63.459,110.633,-0.138,566.05
7,UPS,Industrials,80308183040,United States,us_market,6.021277e+06,1.096,0.557,3.915,66.559,93.485,0.906,559.40
8,PYPL,Financial Services,57867272192,United States,us_market,1.068308e+07,1.433,-0.763,20.668,15.506,65.772,-4.950,536.60
9,ABBV,Healthcare,417597620224,United States,us_market,6.362559e+06,0.352,0.155,11.961,92.433,225.709,2.863,495.15


In [142]:
def find_tier(tiers):
    # return the first tier index where the count (tier[0]) is > 0
    for i, tier in enumerate(tiers):
        if tier[0] > 0:
            return i
    # fallback if no tier met the condition
    return False

def find_percent_tier(percent):
    # simple allocation policy for additional tickers
    if percent <= 5:
        return 2
    elif percent <= 10:
        return 1
    return 0

def make_portfolio(df):
    portfolio = {}
    industry_set = {}

    # Tiers:
    # 4 stocks at 15%
    # 2 stocks at 10%
    # 4 stocks at 5%
    tiers = [(4, 15), (2, 10), (4, 5)]

    for _, row in df.iterrows():

        # Find available tiers, break when portfolio is full
        tier = find_tier(tiers)
        if tier is False:
            break
        percent = tiers[tier][1]

        # Store industry percentages
        if row["Sector"] not in industry_set:
            portfolio[row["Ticker"]] = percent
            industry_set[row["Sector"]] = percent
            tiers[tier] = (tiers[tier][0] - 1, tiers[tier][1])

        else:
            industry_cap = 40 - industry_set[row["Sector"]]
            if industry_cap <= 0:
                    continue
            if percent > industry_cap:
                percent = industry_cap
                tier = find_percent_tier(percent)
            portfolio[row["Ticker"]] = percent
            industry_set[row["Sector"]] += percent
            tiers[tier] = (tiers[tier][0] - 1, tiers[tier][1])


    return portfolio, industry_set
        

In [143]:
portfolio, industries = make_portfolio(filtered_df)
portfolio

{'LLY': 15,
 'BIIB': 15,
 'CAT': 15,
 'AMZN': 15,
 'BMY': 10,
 'BK': 10,
 'UPS': 5,
 'PYPL': 5,
 'BAC': 5,
 'BLK': 5}

In [144]:
industries

{'Healthcare': 40,
 'Industrials': 20,
 'Consumer Cyclical': 15,
 'Financial Services': 25}

In [148]:
start_date = day1
end_date = day15

fx_rate = float(yf.Ticker("CAD=X").history(start=start_date, end=end_date)["Close"].iloc[-1])

portfolio_capital = 1_000_000-(2.15*fx_rate)

records = []
total_stock_value = 0

for ticker, weight in portfolio.items():

    t = yf.Ticker(ticker)
    hist = t.history(start=start_date, end=end_date)

    last_close = hist["Close"].iloc[-1]
    currency = t.info.get("currency", "USD")

    # Convert price to CAD
    if currency == "USD":
        price_cad = last_close * fx_rate
    else:
        price_cad = last_close

    allocation = portfolio_capital * (weight / 100)
    shares = allocation / price_cad

    total_stock_value += price_cad*shares

    value_cad = shares * price_cad

    records.append({
        "Ticker": ticker,
        "Price (native)": last_close,
        "Price (CAD)": price_cad,
        "Currency": currency,
        "Shares": shares, 
        "Value (CAD)": value_cad,                     
        "Weight (%)": weight
    })

# Convert to DF for fee calculation
Portfolio_Final = pd.DataFrame(records)

# ---------------------------------------------------
# Apply portfolio-level fee rule
# ---------------------------------------------------
total_shares = Portfolio_Final["Shares"].sum()

if total_shares > 2150:
    total_fee_usd = 2.15
else:
    total_fee_usd = total_shares * 0.001

total_fee_cad = round(total_fee_usd * fx_rate, 4)

# distribute fee across stocks (or attach same value to all)

# Round outputs
Portfolio_Final["Shares"] = Portfolio_Final["Shares"].round(4)
Portfolio_Final["Value (CAD)"] = Portfolio_Final["Value (CAD)"].round(2)
Portfolio_Final["Price (native)"] = Portfolio_Final["Price (native)"].round(2)
Portfolio_Final["Price (CAD)"] = Portfolio_Final["Price (CAD)"].round(2)
    
# Sort for clean output
Portfolio_Final = Portfolio_Final.sort_values("Weight (%)", ascending=False).reset_index(drop=True)

Portfolio_Final

,Ticker,Price (native),Price (CAD),Currency,Shares,Value (CAD),Weight (%)
0,LLY,1025.28,1439.03,USD,104.2364,149999.55,15
1,BIIB,167.55,235.16,USD,637.8486,149999.55,15
2,CAT,554.03,777.61,USD,192.8985,149999.55,15
3,AMZN,234.69,329.40,USD,455.3732,149999.55,15
4,BMY,46.65,65.48,USD,1527.2816,99999.70,10
5,BK,110.48,155.06,USD,644.8922,99999.70,10
6,UPS,94.34,132.41,USD,377.6112,49999.85,5
7,PYPL,62.67,87.95,USD,568.4795,49999.85,5
8,BAC,52.61,73.84,USD,677.1307,49999.85,5
9,BLK,1057.94,1484.87,USD,33.6728,49999.85,5


In [149]:
rounded_stock_value = 0
for i in range(len(Portfolio_Final)):
    row = Portfolio_Final.iloc[i]

    rounded_stock_value += row['Value (CAD)']
    

total_portfolio_value = total_fee_cad + total_stock_value
rounded_portfolio_value = total_fee_cad + rounded_stock_value
print(f"Rounded Portfolio Value (may exceed $1 million): {rounded_portfolio_value.round(2)}")
print(f"Actual Portfolio Value: {total_portfolio_value.round(2)}")


Rounded Portfolio Value (may exceed $1 million): 1000000.02
Actual Portfolio Value: 1000000.0


In [151]:
stocks_csv = Portfolio_Final[["Ticker", "Shares"]]
stocks_csv.to_csv("Stocks_Group_18.csv",index=False)